# Transfer Learning approach - InceptionResNetV2
In this notebook we have adopted a double finetuning technique based on the InceptionResNetV2 pre-trained model.

Import the needed libraries and connect Drive:

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tensorflow as tf
import numpy as np
import shutil

SEED = 1518
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Get dataset directory
dataset_dir = os.path.join(cwd, 'drive/My Drive/MaskDataset')
dataset_dir
train_dir=os.path.join(dataset_dir,'training')
test_dir = os.path.join(dataset_dir, 'test')
os.listdir(dataset_dir)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='nearest',
                                        rescale=1./255,
                                        validation_split=0.2)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255,
                                        validation_split=0.2)

In [ ]:
# Batch size
bs = 8

# Image shape forced
img_width = 256
img_height = 256

# Classes
num_classes = 3

# Generate Training & Validation
train_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=bs,
                                               classes=None,
                                               class_mode='categorical', ##automatic 1-hot encoding
                                               color_mode='rgb',        
                                               shuffle=True,
                                               subset='training',
                                               seed=SEED) 
valid_gen = train_data_gen.flow_from_directory(train_dir,
                                               batch_size=bs,
                                               classes=None,
                                               class_mode='categorical', ##automatic 1-hot encoding
                                               color_mode='rgb',        
                                               shuffle=True,
                                               subset='validation',
                                               seed=SEED)



# Training set creation (85pc) and valid set creation(15pc)
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_height, img_width, 3], [None, num_classes]))
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_height, img_width, 3], [None, num_classes]))

# Repeat()
train_dataset.repeat()
valid_dataset.repeat()

Load the InceptionResNetV2 model. Loaded weights are the ones of the network trained on the *ImageNet* database. The fully-connected part is removed by setting include_top to False.

In [ ]:
# Load InceptionResNetV2 Model

Inc = tf.keras.applications.InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

First finetuning: layers of the base are freezed until the 50th

In [ ]:
# Create Model

finetuning = True

if finetuning:
    freeze_until = 50
    
    for layer in Inc.layers[:freeze_until]:
        layer.trainable = False
    for layer in Inc.layers[freeze_until:]:
        layer.trainable = True    
else:
    Inc.trainable = False
    
model = tf.keras.Sequential()
model.add(Inc)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer='l2'))
model.add(tf.keras.layers.Dropout(0.55))
model.add(tf.keras.layers.Dense(units=64, activation='relu', kernel_regularizer='l2'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

# Model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 6, 6, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               393472    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 195       
Total params: 54,746,851
Trainable params: 54,211,827
Non-trainable params: 535,024
____________________________________

Set optimization parameters: loss, optimizer and metrics

In [ ]:
# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

In [ ]:
# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

Load tensorboard (uncomment to load)

In [ ]:
#%reload_ext tensorboard
#%tensorboard --logdir /content/drive/My\ Drive/MaskDataset/classification_experiments --port 8008

Set callbacks and visualization of losses and metrics on Tensorboard

In [ ]:
# Callbacks

from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/MaskDataset/', 'classification_experiments_Inc')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN_1'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Visualize Learning on Tensorboard
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# Shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

Apply Early Stopping

In [ ]:
# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [ ]:
# Model fitting

model.fit(x=train_dataset,
          epochs=100,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Epoch 1/100
563/563 [==============================] - 1273s 2s/step - loss: 4.5242 - accuracy: 0.7099 - val_loss: 3.1815 - val_accuracy: 0.8238
Epoch 2/100
563/563 [==============================] - 237s 421ms/step - loss: 2.4884 - accuracy: 0.8405 - val_loss: 2.0498 - val_accuracy: 0.8301
Epoch 3/100
563/563 [==============================] - 233s 414ms/step - loss: 1.5306 - accuracy: 0.8767 - val_loss: 1.1993 - val_accuracy: 0.8986
Epoch 4/100
563/563 [==============================] - 232s 412ms/step - loss: 1.0103 - accuracy: 0.8967 - val_loss: 0.8513 - val_accuracy: 0.8968
Epoch 5/100
563/563 [==============================] - 230s 408ms/step - loss: 0.7219 - accuracy: 0.9180 - val_loss: 0.6878 - val_accuracy: 0.8870
Epoch 6/100
563/563 [==============================] - 229s 408ms/step - loss: 0.5447 - accuracy: 0.9291 - val_loss: 0.6063 - val_accuracy: 0.8879
Epoch 7/100
563/563 [==============================] - 229s 407ms/step - loss: 0.4472 - accuracy: 0.9307 - val_loss: 0.5

Second finetuning: keep patience value to 5 for Early Stopping and freeze layers until the 28th. Now 22 more layers' weights are trained starting from the obtained results. Learning rate is decreased to 1e-5.

In [ ]:
# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)
    
    finetuning = True

if finetuning:
    freeze_until = 28        # layer from which we want to fine-tune
    
    for layer in Inc.layers[:freeze_until]:
        layer.trainable = False
    for layer in Inc.layers[freeze_until:]:
        layer.trainable = True

lr = 1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)   
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)    
model.fit(x=train_dataset,
          epochs=100,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)


Epoch 1/100
563/563 [==============================] - 238s 422ms/step - loss: 0.1189 - accuracy: 0.9736 - val_loss: 0.3744 - val_accuracy: 0.9021
Epoch 2/100
563/563 [==============================] - 233s 414ms/step - loss: 0.0861 - accuracy: 0.9820 - val_loss: 0.3306 - val_accuracy: 0.9173
Epoch 3/100
563/563 [==============================] - 233s 414ms/step - loss: 0.0809 - accuracy: 0.9856 - val_loss: 0.3499 - val_accuracy: 0.9146
Epoch 4/100
563/563 [==============================] - 233s 414ms/step - loss: 0.0668 - accuracy: 0.9898 - val_loss: 0.3660 - val_accuracy: 0.9119
Epoch 5/100
563/563 [==============================] - 236s 419ms/step - loss: 0.0734 - accuracy: 0.9891 - val_loss: 0.3739 - val_accuracy: 0.9164
Epoch 6/100
563/563 [==============================] - 237s 420ms/step - loss: 0.0642 - accuracy: 0.9918 - val_loss: 0.3572 - val_accuracy: 0.9110
Epoch 7/100
563/563 [==============================] - 233s 414ms/step - loss: 0.0622 - accuracy: 0.9902 - val_loss: 0

Third finetuning: unlock 7 more layers to be trained and use half of the previous learning rate. The Patience for the Early Stopping is reduced to 3.

In [ ]:
# Early Stopping

early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    callbacks.append(es_callback)
    
    finetuning = True

if finetuning:
    freeze_until = 21        # layer from which we want to fine-tune
    
    for layer in Inc.layers[:freeze_until]:
        layer.trainable = False
    for layer in Inc.layers[freeze_until:]:
        layer.trainable = True

lr = 0.5*1e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)   
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)    
model.fit(x=train_dataset,
          epochs=100,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)


Epoch 1/100
563/563 [==============================] - 237s 420ms/step - loss: 0.0552 - accuracy: 0.9924 - val_loss: 0.3834 - val_accuracy: 0.9137
Epoch 2/100
563/563 [==============================] - 236s 419ms/step - loss: 0.0485 - accuracy: 0.9940 - val_loss: 0.3738 - val_accuracy: 0.9181
Epoch 3/100
563/563 [==============================] - 236s 419ms/step - loss: 0.0508 - accuracy: 0.9933 - val_loss: 0.3816 - val_accuracy: 0.9199
Epoch 4/100
563/563 [==============================] - 239s 424ms/step - loss: 0.0464 - accuracy: 0.9956 - val_loss: 0.3262 - val_accuracy: 0.9270
Epoch 5/100
563/563 [==============================] - 244s 433ms/step - loss: 0.0458 - accuracy: 0.9951 - val_loss: 0.3535 - val_accuracy: 0.9173
Epoch 6/100
563/563 [==============================] - 238s 423ms/step - loss: 0.0480 - accuracy: 0.9951 - val_loss: 0.3495 - val_accuracy: 0.9146
Epoch 7/100
563/563 [==============================] - 238s 422ms/step - loss: 0.0461 - accuracy: 0.9940 - val_loss: 0

Test the model and export results

In [ ]:
import os
from PIL import Image
cwd = os.getcwd()

dir = os.path.join('/content/drive/My Drive/Progetto/', 'test')
results = {}
for subdir, dirs, files in os.walk(dir):
    for file in files:
        filepath = subdir + os.sep + file
        img = Image.open(filepath).convert('RGB')
        img =img.resize((img_height,img_width))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, 0) 
        img_tensor=tf.convert_to_tensor(img_array)
        img_tensor = tf.cast(img_array,tf.float32)/255.
        softmax=model(img_tensor)
        prediction = np.argmax(softmax)   # predicted class
        results[file] = prediction

In [ ]:
results

In [ ]:
import os
from datetime import datetime

def create_csv(results, results_dir='/content/drive/My Drive/MaskDataset/Results'):

    csv_fname = 'results_inception_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


In [ ]:
create_csv(results)